# Feed Parser from the urls  and Database storage along with categorizing it into four categories and using NLP for categorization 
a)Terrorism / protest / political unrest / riot
b)Positive/Uplifting
c) Natural Disasters
d)Others

## 1. Importing Libraries and Modules

In [1]:
# Import necessary libraries and modules
import feedparser
import hashlib
from datetime import datetime
from sqlalchemy import create_engine, Column, String, DateTime, Integer, ForeignKey, Enum
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm.exc import NoResultFound


## 2. Setting Up RSS Feed URLs

In [2]:
#List of RSS feeds , extracting data from the xml urls 
rss_feeds = ["http://rss.cnn.com/rss/cnn_topstories.rss.xml","https://cms.qz.com/feed/.xml","https://moxie.foxnews.com/google-publisher/politics.xml",
             "https://feeds.feedburner.com/NewshourWorld.xml","https://feeds.bbci.co.uk/news/world/asia/india/rss.xml"]


## 3. Extracting Data from RSS Feeds

In [3]:
 # Loop through each entry in the feed
for feed_url in rss_feeds:
      feed = feedparser.parse(feed_url)

In [4]:
 # Loop through each entry in the feed
for entry in feed.entries:
        title = entry.title
        content = entry.summary
        publication_date = entry.published
        source_url = entry.link

## 4. Cleaning HTML Tags from Content

In [5]:
from bs4 import BeautifulSoup
import re

# Function to clean HTML tags from the content
def clean_html_tags(html_content):
    # Check if html_content is a string or a file path
    if isinstance(html_content, str):
        soup = BeautifulSoup(html_content, 'html.parser')
    else:
        # Assuming html_content is a file path, open the file and pass the filehandle
        with open(html_content, 'r', encoding='utf-8') as file:
            soup = BeautifulSoup(file, 'html.parser')

    cleaned_text = soup.get_text()
    # Additional cleaning if needed
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

# List of RSS feed URLs
rss_feeds = [
    'http://rss.cnn.com/rss/cnn_topstories.rss.xml',
    "https://cms.qz.com/feed/.xml",
    "https://moxie.foxnews.com/google-publisher/politics.xml",
    "https://feeds.feedburner.com/NewshourWorld.xml",
    "https://feeds.bbci.co.uk/news/world/asia/india/rss.xml"
]

# Loop through each RSS feed
for feed_url in rss_feeds:
    feed = feedparser.parse(feed_url)

    # Loop through each entry in the feed
    for entry in feed.entries:
        title = entry.get('title', '')
        content = entry.get('summary', '')
        publication_date = entry.get('published', '')
        source_url = entry.get('link', '')

        # Process the extracted data (e.g., removing HTML tags, cleaning text)
        cleaned_content = clean_html_tags(content)



## 5. Sentiment Analysis and Categorization

In [6]:
import feedparser
from bs4 import BeautifulSoup
import re
import mysql.connector
from mysql.connector import Error
from datetime import datetime
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download the VADER lexicon for sentiment analysis
nltk.download('vader_lexicon')

# Initialize the SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\DHAWA\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


##  6. MySQL Database Configuration

In [7]:
def categorize_news(content):
    # Keywords for each category
    terrorism_keywords = ['terrorism', 'protest', 'unrest', 'riot', 'political']
    uplifting_keywords = ['uplifting', 'positive', 'inspiring']
    natural_disasters_keywords = ['natural disaster', 'earthquake', 'flood', 'hurricane', 'disaster']

    # Check for keywords in the content
    if any(keyword in content.lower() for keyword in terrorism_keywords):
        return 'Terrorism'
    elif any(keyword in content.lower() for keyword in uplifting_keywords):
        return 'Positive'
    elif any(keyword in content.lower() for keyword in natural_disasters_keywords):
        return 'Natural Disasters'
    else:
        return 'Others'


# Function to clean HTML tags from the content
def clean_html_tags(html_content):
    try:
        # Check if html_content is a valid HTML string
        soup = BeautifulSoup(html_content, 'html.parser')
        cleaned_text = soup.get_text()
        # Additional cleaning if needed
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        return cleaned_text
    except Exception as e:
        print(f"Error cleaning HTML tags: {e}")
        return html_content

# MySQL Database Configuration
Db_config = {
    'host': '192.168.1.45',
    'user': 'root',
    'password': 'Shubhang2000@1',
    'database': 'Data_config',
}

## 7. Inserting Data into MySQL Database

In [8]:
# Function to insert data into MySQL database
def insert_into_database(title, content, publication_date, source_url, category):
    try:
        connection = mysql.connector.connect(**Db_config)
        cursor = connection.cursor()

        # Check if publication_date is not empty
        if publication_date:
            try:
                # Convert publication_date to MySQL DATETIME format
                publication_datetime = datetime.strptime(publication_date, '%a, %d %b %Y %H:%M:%S %Z')
            except ValueError as e:
                # Print problematic publication_date values
                print(f"Error parsing publication_date: {publication_date}. Error: {e}")
                publication_datetime = None
        else:
            publication_datetime = None
            
        # Replace 'category_placeholder' with 'Natural Disasters' for 'Others' category
        if category == 'Others':
            category = 'Natural Disasters'


        # Insert new article into the database
        cursor.execute('''
            INSERT INTO news_articles (title, content, publication_date, source_url, category)
            VALUES (%s, %s, %s, %s, %s)
        ''', (title, content, publication_datetime, source_url, category))

        # Commit the changes
        connection.commit()
        print("Data inserted into the database successfully!")

    except Error as e:
        print(f"Error: {e}")

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()

# List of RSS feed URLs
rss_feeds = [
    'http://rss.cnn.com/rss/cnn_topstories.rss.xml',
    "https://cms.qz.com/feed/.xml",
    "https://moxie.foxnews.com/google-publisher/politics.xml",
    "https://feeds.feedburner.com/NewshourWorld.xml",
    "https://feeds.bbci.co.uk/news/world/asia/india/rss.xml"
]


# Loop through each RSS feed
for feed_url in rss_feeds:
    feed = feedparser.parse(feed_url)

    # Loop through each entry in the feed
    for entry in feed.entries:
        title = entry.get('title', '')
        content = entry.get('summary', '')
        publication_date = entry.get('published', '')
        source_url = entry.get('link', '')

        # Process the extracted data (e.g., removing HTML tags, cleaning text)
        cleaned_content = clean_html_tags(content)

        # Categorize news based on sentiment and keywords
        category = categorize_news(cleaned_content)

        # Insert data into MySQL database
        insert_into_database(title, cleaned_content, publication_date, source_url, category)

Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database successfully!
Data inserted into the database su